# Communication Pipeline

In this unit we will gain understanding and practice on how information is passed between different components of the drone such as the onboard computer (Aero Compute Board) and the flight controller (AeroFC).

You should have already been exposed to the basic concepts of the Robot Operating System (ROS) during the online portion of this class. These concepts including things like nodes, topics, messages, services, etc. If you are unsure/uncomfortable with what these concepts imply, there are great tutorials online for walking you through the basics of ROS. Please see:

- https://erlerobotics.com/blog/ros-introduction/
- http://wiki.ros.org/ROS/Tutorials

## Talker - Listener

We are going to put some of basic tools of ROS into practice on the Intel RTF Drone. We are going to start by creating and sending a simple message in one ROS node (i.e. talker) and receiving it in another. This portion relates to [this ROS tutorial](http://wiki.ros.org/ROS/Tutorials/WritingPublisherSubscriber%28python%29), but adapted to our needs.

First make sure you have the most recent codebase on the drone.

_On drone:_

```
cd ~/catkin_ws/src/aero_control
git pull upstream master
cd ~/catkin_ws/src/aero_control/communication_pipeline
```

If you try to run `talker.py` and or `listener.py` you will receive an error stating 

`Exception: CODE INCOMPLETE! Delete this exception and replace with your own code`

### Step 1: Complete Talker

In your preferred text editor, open `talker.py` and replace/fill in the following code block with your own code:

```
'''TODO-START: FILL IN CODE HERE 
* create a string message that contains "Hello World" and the iteration number i
'''
raise Exception("CODE INCOMPLETE! Delete this exception and replace with your own code")
'''TODO-END '''
```

Once complete, you can run the talker node

_Terminal 1:_
```
roscore
```

_Terminal 2:_
```
python talker.py
```

After answering the following questions, you can kill the processes by hitting `Ctrl-c` in each terminal

#### Questions Set 1:

1. List all rosnodes that exist after running talker.py
2. List all rostopics that are being present after running talker.py
3. What command do I run to see what is being published to the `/chatter` topic?
4. What would I change in `talker.py` to make it publish messages more frequently?

### Step 2: Complete Listener

Now we will incorporate a second rosnode to subscribe to the topic created by `talker.py` and listen to the messages passed. Open `listener.py` and replace/fill in the appropriate code block.

Once complete, you can run the talker and listener nodes

_Terminal 1:_
```
roscore
```

_Terminal 2:_
```
python talker.py
```

_Terminal 3:_
```
python listener.py
```

After answering the following questions, you can kill these processes by hitting `Ctrl-C` in each terminal. You will also want to commit the progress you have made so you can later push it to your team's MIT GitHub repositories:

```
cd ~/bwsi-uav/catkin_ws/src/aero_control/communication_pipeline/src
git add -u
git commit -m "filling in code for talker and listener"
```

#### Questions Set 2:

1. List all rosnodes that exist after running talker.py and listener.py
2. List all rostopics that are being present after running talker.py and listener.py
3. In terminal I ran listener.py I see information being printed. Does this mean listner.py is publishing to a topic?

### Step 3: Launch Talker and Listener

It's a cumbersome to be opening a different terminal for every different rosnode you want to start. Instead of doing this, we can use launch files. Before we make a launch file, we need to more properly set up our catkin workspace by turning `aero_control` into a ROS Package and indicate it's dependencies on `std_msgs` and `rospy`

```
cd ~/bwsi-uav/catkin_ws/src
catkin_create_pkg aero_control std_msgs rospy
```

This will create files `~/bwsi-uav/catkin_ws/src/aero_control/CMakeLists.txt` and `~/bwsi-uav/catkin_ws/src/aero_control/package.xml`. You'll want to track these new files on github, so add and commit them so you can later push them to your team's MIT GitHub:

```
cd ~/bwsi-uav/catkin_ws/src/aero_control
git add CMakeLists.txt package.xml
git commit -m 
```


### Step 4: Document and Commit
